<a href="https://colab.research.google.com/github/RabeyaMuna/Skin-Cancer-Detection-Using-CNN/blob/main/Skin_Cancer_Detection_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
# Change directory to the folder containing your dataset
dataset_path = '/content/drive/MyDrive/Skin_Cancer_Dataset'


Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

In [3]:
# Define image size and batch size
image_size = (224, 224)
batch_size = 32
# Create ImageDataGenerator for train set with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create ImageDataGenerator for test and validation sets without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
dataset_path

'/content/drive/MyDrive/Skin_Cancer_Dataset'

In [5]:
# Load train set images
import os

# Define file paths
train_dir = os.path.join(dataset_path, 'train')

# Load train set images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 1934 images belonging to 2 classes.


In [6]:
# Load test set images
test_dir = os.path.join(dataset_path, 'test')

# Load train set images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 932 images belonging to 2 classes.


In [7]:
# Load validation set images
val_dir = os.path.join(dataset_path, 'val')
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator

Found 1864 images belonging to 2 classes.


In [8]:
labels = ['benign', 'malignant']
label_np=np.array(labels)

In [9]:
# Create CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit_generator(generator= train_generator, 
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

<ipython-input-10-ffe23de8475e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator= train_generator,


Epoch 1/5
60/60 [==============================] - 600s 10s/step - loss: 0.0136 - accuracy: 0.9837 - val_loss: 138.8089 - val_accuracy: 0.5248
Epoch 2/5
60/60 [==============================] - 193s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 139.9214 - val_accuracy: 0.5237
Epoch 3/5
60/60 [==============================] - 200s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 139.9184 - val_accuracy: 0.5237
Epoch 4/5
60/60 [==============================] - 193s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 140.2320 - val_accuracy: 0.5226
Epoch 5/5
60/60 [==============================] - 193s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 139.6372 - val_accuracy: 0.5248


In [11]:
test_generator.reset()  # Reset the generator before evaluation
scores = model.evaluate_generator(generator=test_generator, steps=test_generator.samples // batch_size)

# Extract the accuracy from the evaluation scores
accuracy = scores[1]  # Assuming accuracy is the second metric in the list of scores

print("Test Accuracy: {:.2f}%".format(accuracy * 100))

<ipython-input-11-68dadb747636>:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(generator=test_generator, steps=test_generator.samples // batch_size)


Test Accuracy: 52.48%
